## 1. Load Data

#### 1.1 Import images

In [1]:
import os
import cv2
import numpy as np

path_img_src = os.path.join(os.getcwd(), 'data/processed')
valid_images = [".jpg",".gif",".png",".tga"]
x = []
for file in os.listdir(path_img_src):
    ext = os.path.splitext(file)[1]
    if ext.lower() not in valid_images: continue
    img = cv2.imread(os.path.join(path_img_src,file), cv2.IMREAD_GRAYSCALE)
    x.append(img)
x = np.array(x)
x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2])
x.shape

(1481, 1, 75, 100)

#### 1.2 Import labels

In [2]:
path_labels_src = os.path.join(os.getcwd(), 'data')
y = []
with open(os.path.join(path_labels_src, 'labels.txt'), "r") as labels_file:
    for line in labels_file:
        y.append(int(line.rstrip('\n')))
y = np.array(y[:x.shape[0]]).reshape(-1,1)
y.shape

(1481, 1)

## 2. Preprocess Data

#### 2.1 Normalize images

In [3]:
x = x / 255
np.min(x), np.max(x)

(0.0, 1.0)

#### 2.2 Convert labels to one-hot encoding

In [4]:
from keras.utils import to_categorical
number_of_classes = 9
print("Before: ", y[0])
y = to_categorical(y, num_classes=number_of_classes)
print("After:  ", y[0])

F:\Python\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Before:  [5]
After:   [0. 0. 0. 0. 0. 1. 0. 0. 0.]


#### 2.3 Split data for train and test

In [5]:
seed = 1
np.random.seed(seed)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=np.random)
print("Train Data | Test Data")
print(("{0:^10} | {1:^10}").format(x_train.shape[0], x_test.shape[0]))

Train Data | Test Data
   1184    |    297    


## 3. Build Model

#### 3.1 Import Keras and use NCHW mode

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.utils import np_utils

from keras import backend
backend.set_image_dim_ordering('th')

#### 3.2 Create reusable functions

In [7]:
c = x.shape[1]
h = x.shape[2]
w = x.shape[3]
no_of_classes = y.shape[1]
print(x.shape, y.shape)

def train_model(model, epochs=100, batch_size=125):
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

def evaluate_model(model):
    print("\n\n")
    scores = model.evaluate(x_test, y_test)
    print("Accuracy: ", scores[1]*100, "%")
    return scores

(1481, 1, 75, 100) (1481, 9)


#### 3.3 Create a baseline model

In [8]:
def baseline_model():
    model = Sequential()
    
    model.add(Conv2D(32, (5, 5), kernel_initializer='normal', activation='relu', input_shape=(c, h, w)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(128, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(no_of_classes, kernel_initializer='normal', activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = baseline_model()
train_model(model)
evaluate_model(model)

Epoch 1/100
1184/1184 [==============================] - 2s 2ms/step - loss: 1.9019 - acc: 0.3784
Epoch 2/100
1184/1184 [==============================] - 0s 298us/step - loss: 1.4563 - acc: 0.4215
Epoch 3/100
1184/1184 [==============================] - 0s 288us/step - loss: 1.3511 - acc: 0.4823
Epoch 4/100
1184/1184 [==============================] - 0s 295us/step - loss: 1.3147 - acc: 0.4932
Epoch 5/100
1184/1184 [==============================] - 0s 293us/step - loss: 1.2947 - acc: 0.5422
Epoch 6/100
1184/1184 [==============================] - 0s 296us/step - loss: 1.2289 - acc: 0.5946
Epoch 7/100
1184/1184 [==============================] - 0s 309us/step - loss: 1.1882 - acc: 0.5853
Epoch 8/100
1184/1184 [==============================] - 0s 297us/step - loss: 1.1432 - acc: 0.6166
Epoch 9/100
1184/1184 [==============================] - 0s 294us/step - loss: 1.0881 - acc: 0.6377 0s - loss: 1.1275 - acc: 0.6
Epoch 10/100
1184/1184 [==============================] - 0s 295us/step -

Epoch 82/100
1184/1184 [==============================] - 0s 298us/step - loss: 0.0615 - acc: 0.9780
Epoch 83/100
1184/1184 [==============================] - 0s 295us/step - loss: 0.0604 - acc: 0.9823
Epoch 84/100
1184/1184 [==============================] - 0s 285us/step - loss: 0.0630 - acc: 0.9823
Epoch 85/100
1184/1184 [==============================] - 0s 296us/step - loss: 0.0573 - acc: 0.9823
Epoch 86/100
1184/1184 [==============================] - 0s 291us/step - loss: 0.0695 - acc: 0.9797
Epoch 87/100
1184/1184 [==============================] - 0s 288us/step - loss: 0.0717 - acc: 0.9789
Epoch 88/100
1184/1184 [==============================] - 0s 294us/step - loss: 0.0584 - acc: 0.9806
Epoch 89/100
1184/1184 [==============================] - 0s 293us/step - loss: 0.0549 - acc: 0.9814
Epoch 90/100
1184/1184 [==============================] - 0s 311us/step - loss: 0.0627 - acc: 0.9738
Epoch 91/100
1184/1184 [==============================] - 0s 299us/step - loss: 0.0600 - ac

[1.4109869348481046, 0.7542087546101323]

## 4. Saving and loading model architecture and weights

#### 4.1 Define reusable functions

In [9]:
import h5py
from keras.models import model_from_json
from keras.models import load_model

path_models = os.path.join(os.getcwd(), 'models')
valid_model_files = [".h5", ".json"]
weights_suffix = '_weights.h5'
architecture_suffix = '_architecture.json'

In [10]:
def get_last_file_number(path):
    numbers = [-1]
    for file in os.listdir(path):
        filename = os.path.splitext(file)[0]
        ext = os.path.splitext(file)[1]
        if ext.lower() not in valid_model_files: continue
        if filename.startswith('model_'): 
            numbers.append(int(''.join(list(filter(str.isdigit, filename)))))
    counter = max(numbers)
    return counter

#### 4.2 Save current model

In [11]:
counter = 0
if not os.path.isdir(path_models): os.makedirs(path_models)
else: counter = get_last_file_number(path_models) + 1

model_name = 'model_' + '{0:03d}'.format(counter)
model_arch_file = path_models + '\\' + model_name + architecture_suffix
model_weights_file = path_models + '\\' + model_name + weights_suffix
with open(model_arch_file, 'w+') as json_file:
    json_file.write(model.to_json(indent=4))
model.save_weights(model_weights_file)

model.save(path_models + '\\' + model_name + '.h5')

print("Saving files:\n\t" + model_name + architecture_suffix + '\n\t' + model_name + weights_suffix)

Saving files:
	model_002_architecture.json
	model_002_weights.h5


#### 4.3 Load model from file

In [12]:
counter = get_last_file_number(path_models)
    
model_name = 'model_' + '{0:03d}'.format(counter)
model_arch_file = path_models + '\\' + model_name + architecture_suffix
model_weights_file = path_models + '\\' + model_name + architecture_suffix

print("Loading files:\n\t" + model_name + architecture_suffix + '\n\t' + model_name + weights_suffix)

if not os.path.isfile(model_arch_file):
    print('Could not find', model_arch_file)
elif not os.path.isfile(model_weights_file):
    print('Could not find', model_weights_file)
else:
#     with open(model_arch_file, 'r') as json_file:
#         model_loaded = model_from_json(json_file.read())
#     model_loaded.load_weights(model_weights_file)
    model_loaded = load_model(path_models + '\\' + model_name + '.h5')

Loading files:
	model_002_architecture.json
	model_002_weights.h5


In [13]:
evaluate_model(model_loaded)




297/297 [==============================] - 0s 582us/step
Accuracy:  75.42087546101322 %


[1.4109869348481046, 0.7542087546101323]

In [25]:
np.argmax(model.predict(x[0][:, None]))

5

In [28]:
sample = np.zeros((75,100))
sample.shape

(75, 100)

In [32]:
sample[None, None, :].shape

(1, 1, 75, 100)